# Load, Fuel Mix, and LMP Data

This notebook walk through how to use `gridstatus` to access to the data availabe on [OASIS](http://oasis.caiso.com/).

While we will be using CAISO in this example, most of this API will also work with all other ISOs.

In [1]:
import gridstatus
import pandas as pd
import os

os.chdir(r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project")


In [ ]:
import os
import time
import datetime
import requests
from tqdm import tqdm
from dateutil.relativedelta import relativedelta

# === Configuration ===
output_dir = r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project\CAISO_REN_DAM_Forecasts"
os.makedirs(output_dir, exist_ok=True)

base_url = "https://oasis.caiso.com/oasisapi/SingleZip"
query_template = (
    "?resultformat=6&queryname=SLD_REN_FCST&version=1&market_run_id=DAM"
    "&startdatetime={start}&enddatetime={end}"
)

# === Date generator ===
def month_ranges(start_date, end_date):
    """Yield (start, end) pairs for each month between start_date and end_date."""
    current = start_date
    while current < end_date:
        next_month = current + relativedelta(months=1)
        yield current, min(next_month, end_date)
        current = next_month

# === Download function with retry ===
def download_file(url, file_path, max_retries=5):
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url, timeout=60)
            if response.status_code == 200 and response.content:
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ Saved: {os.path.basename(file_path)}")
                return True
            elif response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"⚠️ Rate limited (429), retrying in {wait_time}s...")
                time.sleep(wait_time)
                attempt += 1
            else:
                print(f"⚠️ Failed ({response.status_code}) for {os.path.basename(file_path)}")
                return False
        except Exception as e:
            print(f"❌ Error: {e}, retrying...")
            time.sleep(2 ** attempt)
            attempt += 1
    print(f"❌ Giving up after {max_retries} attempts: {os.path.basename(file_path)}")
    return False

# === Main loop ===
start_date = datetime.datetime(2022, 7, 1)
end_date = datetime.datetime(2025, 10, 1)

for start, end in tqdm(list(month_ranges(start_date, end_date)), desc="📅 Fetching monthly ZIPs"):
    start_str = start.strftime("%Y%m%dT07:00-0000")
    end_str = (end - datetime.timedelta(days=1)).strftime("%Y%m%dT07:00-0000")
    url = f"{base_url}{query_template.format(start=start_str, end=end_str)}"

    file_name = f"CAISO_DAM_Forecast_{start.strftime('%Y_%m')}.zip"
    file_path = os.path.join(output_dir, file_name)

    if os.path.exists(file_path):
        print(f"✅ Already exists: {file_name}")
        continue

    print(f"\n📦 Downloading {file_name}...")
    success = download_file(url, file_path)
    if not success:
        print(f"⚠️ Skipped {file_name} after retries.")
    
    # Polite delay to avoid rapid re-requests
    time.sleep(2)


📅 Fetching monthly ZIPs:   0%|          | 0/39 [00:00<?, ?it/s]


📦 Downloading CAISO_DAM_Forecast_2022_07.zip...
✅ Saved: CAISO_DAM_Forecast_2022_07.zip


📅 Fetching monthly ZIPs:   3%|▎         | 1/39 [00:02<01:33,  2.47s/it]


📦 Downloading CAISO_DAM_Forecast_2022_08.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
⚠️ Rate limited (429), retrying in 4s...
✅ Saved: CAISO_DAM_Forecast_2022_08.zip


📅 Fetching monthly ZIPs:   5%|▌         | 2/39 [00:12<04:14,  6.89s/it]

...

📅 Fetching monthly ZIPs:  95%|█████████▍| 37/39 [03:43<00:11,  5.98s/it]


📦 Downloading CAISO_DAM_Forecast_2025_08.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
✅ Saved: CAISO_DAM_Forecast_2025_08.zip


📅 Fetching monthly ZIPs:  97%|█████████▋| 38/39 [03:49<00:06,  6.01s/it]


📦 Downloading CAISO_DAM_Forecast_2025_09.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
✅ Saved: CAISO_DAM_Forecast_2025_09.zip


📅 Fetching monthly ZIPs: 100%|██████████| 39/39 [03:55<00:00,  6.04s/it]


In [ ]:
import os
import time
import datetime
import requests
from tqdm import tqdm
from dateutil.relativedelta import relativedelta

# === Configuration ===
output_dir = r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project\data\CAISO_DAM_Load_Forecasts"
os.makedirs(output_dir, exist_ok=True)

base_url = "https://oasis.caiso.com/oasisapi/SingleZip"
query_template = (
    "?resultformat=6&queryname=SLD_FCST&version=1&market_run_id=DAM"
    "&startdatetime={start}&enddatetime={end}"
)

# === Date generator ===
def month_ranges(start_date, end_date):
    """Yield (start, end) pairs for each month between start_date and end_date."""
    current = start_date
    while current < end_date:
        next_month = current + relativedelta(months=1)
        yield current, min(next_month, end_date)
        current = next_month

# === Download function with retry ===
def download_file(url, file_path, max_retries=5):
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url, timeout=60)
            if response.status_code == 200 and response.content:
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ Saved: {os.path.basename(file_path)}")
                return True
            elif response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"⚠️ Rate limited (429), retrying in {wait_time}s...")
                time.sleep(wait_time)
                attempt += 1
            else:
                print(f"⚠️ Failed ({response.status_code}) for {os.path.basename(file_path)}")
                return False
        except Exception as e:
            print(f"❌ Error: {e}, retrying...")
            time.sleep(2 ** attempt)
            attempt += 1
    print(f"❌ Giving up after {max_retries} attempts: {os.path.basename(file_path)}")
    return False

# === Main loop ===
start_date = datetime.datetime(2022, 7, 1)
end_date = datetime.datetime(2025, 10, 1)

for start, end in tqdm(list(month_ranges(start_date, end_date)), desc="📅 Fetching monthly ZIPs"):
    start_str = start.strftime("%Y%m%dT07:00-0000")
    end_str = (end - datetime.timedelta(days=1)).strftime("%Y%m%dT07:00-0000")
    url = f"{base_url}{query_template.format(start=start_str, end=end_str)}"

    file_name = f"CAISO_DAM_Load_Forecast_{start.strftime('%Y_%m')}.zip"
    file_path = os.path.join(output_dir, file_name)

    if os.path.exists(file_path):
        print(f"✅ Already exists: {file_name}")
        continue

    print(f"\n📦 Downloading {file_name}...")
    success = download_file(url, file_path)
    if not success:
        print(f"⚠️ Skipped {file_name} after retries.")
    
    # Polite delay to avoid rapid re-requests
    time.sleep(2)

📅 Fetching monthly ZIPs:   0%|          | 0/39 [00:00<?, ?it/s]


📦 Downloading CAISO_DAM_Load_Forecast_2022_07.zip...
✅ Saved: CAISO_DAM_Load_Forecast_2022_07.zip


📅 Fetching monthly ZIPs:   3%|▎         | 1/39 [00:02<01:50,  2.90s/it]


📦 Downloading CAISO_DAM_Load_Forecast_2022_08.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
✅ Saved: CAISO_DAM_Load_Forecast_2022_08.zip


📅 Fetching monthly ZIPs:   5%|▌         | 2/39 [00:08<02:53,  4.70s/it]

...

📅 Fetching monthly ZIPs:  95%|█████████▍| 37/39 [03:43<00:11,  5.79s/it]


📦 Downloading CAISO_DAM_Load_Forecast_2025_08.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
✅ Saved: CAISO_DAM_Load_Forecast_2025_08.zip


📅 Fetching monthly ZIPs:  97%|█████████▋| 38/39 [03:49<00:05,  5.85s/it]


📦 Downloading CAISO_DAM_Load_Forecast_2025_09.zip...
⚠️ Rate limited (429), retrying in 1s...
⚠️ Rate limited (429), retrying in 2s...
✅ Saved: CAISO_DAM_Load_Forecast_2025_09.zip


📅 Fetching monthly ZIPs: 100%|██████████| 39/39 [03:56<00:00,  6.07s/it]


In [2]:
import os
import time
import datetime
import requests
from tqdm import tqdm
from dateutil.relativedelta import relativedelta

# === Configuration ===
output_dir = r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project\data\CAISO_LMP_Forecasts"
os.makedirs(output_dir, exist_ok=True)

base_url = "https://oasis.caiso.com/oasisapi/SingleZip"
query_template = (
    "?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM"
    "&startdatetime={start}&enddatetime={end}"
    "&node=TH_SP15_GEN-APND,TH_NP15_GEN-APND,TH_ZP26_GEN-APND"
)

# === Date generator ===
def month_ranges(start_date, end_date):
    """Yield (start, end) pairs for each month between start_date and end_date."""
    current = start_date
    while current < end_date:
        next_month = current + relativedelta(months=1)
        yield current, min(next_month, end_date)
        current = next_month

# === Download function with retry ===
def download_file(url, file_path, max_retries=5):
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url, timeout=60)
            if response.status_code == 200 and response.content:
                with open(file_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ Saved: {os.path.basename(file_path)}")
                return True
            elif response.status_code == 429:
                wait_time = 2 ** attempt
                print(f"⚠️ Rate limited (429), retrying in {wait_time}s...")
                time.sleep(wait_time)
                attempt += 1
            else:
                print(f"⚠️ Failed ({response.status_code}) for {os.path.basename(file_path)}")
                return False
        except Exception as e:
            print(f"❌ Error: {e}, retrying...")
            time.sleep(2 ** attempt)
            attempt += 1
    print(f"❌ Giving up after {max_retries} attempts: {os.path.basename(file_path)}")
    return False

# === Main loop ===
start_date = datetime.datetime(2022, 7, 1)
end_date = datetime.datetime(2025, 10, 1)

for start, end in tqdm(list(month_ranges(start_date, end_date)), desc="📅 Fetching monthly ZIPs"):
    start_str = start.strftime("%Y%m%dT07:00-0000")
    end_str = (end - datetime.timedelta(days=1)).strftime("%Y%m%dT07:00-0000")
    url = f"{base_url}{query_template.format(start=start_str, end=end_str)}"

    file_name = f"CAISO_LMP_Forecast_{start.strftime('%Y_%m')}.zip"
    file_path = os.path.join(output_dir, file_name)

    if os.path.exists(file_path):
        print(f"✅ Already exists: {file_name}")
        continue

    print(f"\n📦 Downloading {file_name}...")
    success = download_file(url, file_path)
    if not success:
        print(f"⚠️ Skipped {file_name} after retries.")
    
    # Polite delay to avoid rapid re-requests
    time.sleep(2)

📅 Fetching monthly ZIPs:   0%|          | 0/39 [00:00<?, ?it/s]


📦 Downloading CAISO_LMP_Forecast_2022_07.zip...
✅ Saved: CAISO_LMP_Forecast_2022_07.zip


📅 Fetching monthly ZIPs:   3%|▎         | 1/39 [00:08<05:26,  8.60s/it]


📦 Downloading CAISO_LMP_Forecast_2022_08.zip...
✅ Saved: CAISO_LMP_Forecast_2022_08.zip


📅 Fetching monthly ZIPs:   5%|▌         | 2/39 [00:17<05:25,  8.79s/it]

...

📅 Fetching monthly ZIPs:  95%|█████████▍| 37/39 [05:28<00:17,  8.84s/it]


📦 Downloading CAISO_LMP_Forecast_2025_08.zip...
✅ Saved: CAISO_LMP_Forecast_2025_08.zip


📅 Fetching monthly ZIPs:  97%|█████████▋| 38/39 [05:38<00:08,  8.99s/it]


📦 Downloading CAISO_LMP_Forecast_2025_09.zip...
✅ Saved: CAISO_LMP_Forecast_2025_09.zip


📅 Fetching monthly ZIPs: 100%|██████████| 39/39 [05:46<00:00,  8.88s/it]
